Oblv Client is from our PyOblv package.
ds_abc is the PySyft Client for the Data Scientist on domain abc
ds_xyz is the PySyft Client for the Data Scientist on domain xyz

In [ ]:
%load_ext autoreload
%autoreload 2
import syft as sy
import oblv
canada = sy.login(url="localhost",port=8081,email="sheldon@caltech.edu",password="changethis")
italy = sy.login(url="localhost",port=8082,email="sheldon@caltech.edu",password="changethis")

In [ ]:
#Oblivious client facilitates Secure Enclave Deployment
oblv_client = oblv.authenticate("your_api_key")

In [ ]:
depl = sy.create_deployment(oblv_client,[canada,italy],"demo_openmined","first",infra="r5.xlarge")


In [ ]:
depl.deployment_id

In [ ]:
# depl = sy.create_deployment(oblv_client,[ds_abc,ds_xyz],"demo_openmined","first",infra="m5.2xlarge")
# depl = sy.oblv.model.DeploymentClient(deployment_id="d-4f7dddd2-5c3f-4beb-9eaa-9e97ecfc205a",oblv_client=oblv_client,domain_clients=[ds_abc,ds_xyz],user_key_name="first") #connection_port key can be added to set the port on which oblv_proxy will run
# depl.deployment_id

In [ ]:
depl_info = oblv_client.deployment_info(depl.deployment_id)
print(depl_info.current_state)

In [ ]:
canada_ptr = canada.store[-1]
italy_ptr = italy.store[-1]

In [ ]:
depl.__dict__

tensor_abc and tensor_xyz are the two tensors selected from two different domains.

Creating a deploynent now, using both the domains (abc and xyz).

The public-private key pair was already generated for both these domains, but if it is not, the admins of the domain can use the method **create_key** in oblv api to create one.

Deployment with shown id is created. The status of the deployment can be checked with the next block. Once the status is **Running**, the further steps can be taken after an initialization of the connection. 

The port on which the connection is to be made can be specified either during the time of creation of deployment or can be later updated as shown below.

*Do not change the connection port after initializing the connection*

In [ ]:
canada.oblv.check_connection(depl)

In [ ]:
depl.check_proxy_running()

In [ ]:
depl.initiate_connection(3032)

In [ ]:
canada_dataset = canada.oblv.send_dataset(deployment=depl,dataset=canada_ptr)
italy_dataset = italy.oblv.send_dataset(deployment=depl,dataset=italy_ptr)

In [ ]:
depl.get_uploaded_datasets()

In [ ]:
type(canada_dataset)

In [ ]:
dataset_result = canada_dataset + italy_dataset
# dataset_result = sy.OblvTensorWrapper(id="r-3533056d-e408-40bc-8550-d6d98d7cfca3",deployment_client=depl)

In [ ]:
# import numpy as np
# arr = np.array([1,2,3,4,5])
# dataset_result = dataset_abc + arr

In [ ]:
dataset_sum_result = dataset_result.sum(keepdims=True)
# dataset_result = sy.OblvTensorWrapper(id="r-3533056d-e408-40bc-8550-d6d98d7cfca3",deployment_client=depl)

In [ ]:
dataset_sum_result.id

In [ ]:
# p_id = depl.request_publish(dataset_sum_result.id, sigma=0.5)
p_id=dataset_sum_result.request_publish(sigma=1000)

print(p_id)

In [ ]:
print(canada.privacy_budget)
print(italy.privacy_budget)

In [ ]:
depl.check_publish_request_status(p_id)

In [ ]:
print(canada.privacy_budget)
print(italy.privacy_budget)

In [ ]:
result = depl.fetch_result(p_id)
print(result)

In [ ]:
depl.close_connection()
oblv_client.remove_deployment(depl.deployment_id)

In [ ]:
italy.oblv.get_key()

In [ ]:
depl.fetch_current_proxy_logs()

Known Issues - 

Large dataset cannot be used as of now, cause they go in long re-runs for cache calculation.

Proxy sends 200 status code while the enclave returns 400.

Mean method when performed on tensor returns a tensor with data_subjects not of type of array and it fails when we perform publish on it.
